In [1]:
from auto_example import handle_transforms
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import statistics
import numpy

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [2]:
from sklearn.datasets import fetch_openml
dataset      = fetch_openml('houses', version=1) # name is dataset_name
X            = dataset['data']
target       = dataset['target'].to_frame()

print(X)
print(target)
print(dataset['target_names'])

/home/ksrinivs/anaconda3/envs/expressions/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


       median_income  housing_median_age  total_rooms  total_bedrooms  \
0             8.3252                41.0        880.0           129.0   
1             8.3014                21.0       7099.0          1106.0   
2             7.2574                52.0       1467.0           190.0   
3             5.6431                52.0       1274.0           235.0   
4             3.8462                52.0       1627.0           280.0   
...              ...                 ...          ...             ...   
20635         1.5603                25.0       1665.0           374.0   
20636         2.5568                18.0        697.0           150.0   
20637         1.7000                17.0       2254.0           485.0   
20638         1.8672                18.0       1860.0           409.0   
20639         2.3886                16.0       2785.0           616.0   

       population  households  latitude  longitude  
0           322.0       126.0     37.88    -122.23  
1          2401.0

In [3]:
# Set standard classifier (could be any AutoML as well)
estimator = RandomForestRegressor(random_state = 1908)
# Evaluate on original data
scores = cross_val_score(estimator, X, numpy.ravel(target), cv=3, scoring='r2')
print("Averaged r2 score on original data:  " + str(statistics.mean(scores)))

Averaged r2 score on original data:  0.5814812832363954


In [4]:
cols = X.columns
print(cols)

Index(['median_income', 'housing_median_age', 'total_rooms', 'total_bedrooms',
       'population', 'households', 'latitude', 'longitude'],
      dtype='object')


In [5]:
import requests
import json

buf = ['houses']
for i in cols:
    buf.append(i)
query = ' '.join(buf)
print('query:' + query)
        
rest_url = 'http://localhost:8001/api/github_search'
payload = {
    "query": query,
}
ret = requests.get(rest_url, params=payload)
        
data = json.loads(ret.content)
        
urls = [d['URL'].replace('github.com', 'raw.githubusercontent.com').replace('/blob/', '/') for d in data]

print(urls)


query:houses median_income housing_median_age total_rooms total_bedrooms population households latitude longitude
['https://raw.githubusercontent.com/dan-a-iancu/airm/0a5d14e19bd726ff24c2decf4ad70ca1ddc7c041/ML_Pipeline/S03_04_Descriptive_to_Predictive_Analytics.ipynb', 'https://raw.githubusercontent.com/binarybrain-009/M.L.-learning-and-reading-material/4d119e1d49bbfe8427645ae4090114bc5b1acf1f/01_end_to_end_machine_learning_project-checkpoint.ipynb', 'https://raw.githubusercontent.com/sudhanshu1402/Basic-Machine-Learning-Projects/a07146832e2289f1b9b9924a0806f74e63c375b6/Housing%20Prices%20Prediction/Housing%20Prices%20Prediction.ipynb', 'https://raw.githubusercontent.com/CS196Illinois/Group33-FA22/52fc391f2344df3c7c1d62ada34fe09e30dd5a56/Project/Backend/price_prediction_models.ipynb', 'https://raw.githubusercontent.com/devilsAXE/csd-data-analytics-with-python-course/4fc6181b00bbebf541b9cb82932b6b92bc728b57/Lab/LAB-2/01_end_to_end_machine_learning_project.ipynb', 'https://raw.githubuse

In [16]:
import urllib
import logging
import nbformat
from nbconvert.exporters import PythonExporter

def get_python(nbhandle):
    # read source notebook
    print('reading:' + nbhandle)
    f = urllib.request.urlopen(nbhandle)
    try:
        nb = nbformat.read(f, as_version=4)
        python_exporter = PythonExporter()
        code = python_exporter.from_notebook_node(nb)
        return code[0]
    except:
        # logging.exception("message")
        return None


In [18]:
expressions = []
code2count = {}

for url in urls:
    code = get_python(url)
    if code is None:
        continue
    req = {'repo': url, 'source': code, 'indexName': 'expressions'}
    response = requests.post('http://localhost:4567/index', json=req)
    for r in response.json():
        if r['code'] not in code2count:
            code2count[r['code']] = 0
        code2count[r['code']] += 1

codes = {k: v for k, v in sorted(code2count.items(), reverse=True, key=lambda item: item[1])[:10]}

for idx, c in enumerate(codes):
    expressions.append({'expr_name': 'expr' + str(idx), 'code': c})

print(expressions)


reading:https://raw.githubusercontent.com/dan-a-iancu/airm/0a5d14e19bd726ff24c2decf4ad70ca1ddc7c041/ML_Pipeline/S03_04_Descriptive_to_Predictive_Analytics.ipynb
reading:https://raw.githubusercontent.com/binarybrain-009/M.L.-learning-and-reading-material/4d119e1d49bbfe8427645ae4090114bc5b1acf1f/01_end_to_end_machine_learning_project-checkpoint.ipynb
reading:https://raw.githubusercontent.com/sudhanshu1402/Basic-Machine-Learning-Projects/a07146832e2289f1b9b9924a0806f74e63c375b6/Housing%20Prices%20Prediction/Housing%20Prices%20Prediction.ipynb
reading:https://raw.githubusercontent.com/CS196Illinois/Group33-FA22/52fc391f2344df3c7c1d62ada34fe09e30dd5a56/Project/Backend/price_prediction_models.ipynb
reading:https://raw.githubusercontent.com/devilsAXE/csd-data-analytics-with-python-course/4fc6181b00bbebf541b9cb82932b6b92bc728b57/Lab/LAB-2/01_end_to_end_machine_learning_project.ipynb
reading:https://raw.githubusercontent.com/arunpeddakotla/hansonml2/72501d47132a18da11e741db27c30c3061c1b9f0/2.%2

In [21]:
# Analyze given transforms and if applicable create SKLEARN Function Transforms as a pipeline
transforms_suggested, correlation = handle_transforms("both", expressions, target, X, 'houses.csv')

------------------------------------------------------
Dataset columns (X): ['median_income', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'latitude', 'longitude', 'expr5', 'expr6', 'expr7', 'expr8', 'expr9']
Dataset columns (Y): ['median_house_value']
Traceback (most recent call last):
  File "/home/ksrinivs/anaconda3/envs/expressions/lib/python3.10/site-packages/pandas/core/indexes/base.py", line 3803, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'income_cat'

The above exception was the direct cause of the following exception:

Traceba

In [22]:
# Pipeline of suggested transforms (as a SKLEARN functional transformer)
transforms_suggested

[('expr5',
  FunctionTransformer(func=<function wrapper_func.<locals>.df_func at 0x7fd4702f0af0>)),
 ('expr6',
  FunctionTransformer(func=<function wrapper_func.<locals>.df_func at 0x7fd4704e32e0>)),
 ('expr7',
  FunctionTransformer(func=<function wrapper_func.<locals>.df_func at 0x7fd4704e3370>)),
 ('expr8',
  FunctionTransformer(func=<function wrapper_func.<locals>.df_func at 0x7fd4704e3520>)),
 ('expr9',
  FunctionTransformer(func=<function wrapper_func.<locals>.df_func at 0x7fd4704e3c70>))]

In [23]:
# Add estimator to suggested transformation pipeline
transforms_suggested.append(('estimator', estimator))
pipeline = Pipeline(transforms_suggested)

In [24]:
# Evaluate with data augmentation added as function transformers based on original data
scores = cross_val_score(pipeline, X, numpy.ravel(target), cv=3, scoring='r2')
print("Averaged r2 score with augmentations on original data:  " + str(statistics.mean(scores)))

Averaged r2 score with augmentations on original data:  0.6532869739725201
